In [1]:
import pandas as pd
import geopandas as gpd

/home/data/work/Fatemeh/DEiA-2-main/.direnv/python-3.10.1/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
CRS = "EPSG:32733"

In [3]:
facilities = pd.read_csv('nl_osm_locations.csv')
facilities = gpd.GeoDataFrame(
    facilities, geometry=gpd.points_from_xy(facilities['lon'], facilities['lat'], crs=CRS)
).drop(columns=['lat', 'lon'])

In [4]:
neighborhoods = pd.read_csv('output_list.csv')
neighborhoods = gpd.GeoDataFrame(
    neighborhoods, geometry=gpd.points_from_xy(neighborhoods['longitude'], neighborhoods['latitude']), crs=CRS
).drop(columns=['latitude', 'longitude'])
neighborhoods

,neighbourhood,municipality,play_demand,a_00_14,geometry
0,Appingedam-Oost,Appingedam,7,900,POINT (6.859 53.307)
1,Hoogkerk-Zuid,Groningen,5,965,POINT (6.492 53.206)
2,Beijum-West,Groningen,6,1005,POINT (6.587 53.249)
3,Beijum-Oost,Groningen,6,1175,POINT (6.601 53.249)
4,Haren,Groningen,6,1485,POINT (6.586 53.171)
...,...,...,...,...,...
486,Numansdorp Woonkern,Hoeksche Waard,6,1110,POINT (4.438 51.735)
487,Bedum,Het Hogeland,6,1205,POINT (6.600 53.302)
488,Marum,Westerkwartier,5,910,POINT (6.267 53.146)
489,Zuidhorn,Westerkwartier,6,1475,POINT (6.408 53.243)


In [5]:
def join_nearest_type(type_):
    return gpd.sjoin_nearest(neighborhoods, facilities.loc[facilities['type'] == type_], distance_col='distance')

In [6]:
def join_nearest_types(types):
    dfs = [join_nearest_type(t) for t in types]
    return pd.concat(dfs, axis=0, ignore_index=True)

In [7]:
facility_types = list(facilities['type'].unique())
facility_types

['nl_sports_centre',
 'nl_sports_hall',
 'nl_kindergarten',
 'nl_childcare',
 'nl_fitness_station',
 'nl_pitch',
 'nl_school',
 'nl_swimming_pool',
 'nl_stadium',
 'nl_college',
 'nl_fitness_centre',
 'nl_university',
 'nl_nursing_house',
 'nl_park',
 'nl_golf_course',
 'nl_horse_riding',
 'nl_track']

In [8]:
joined = join_nearest_types(facility_types)
joined

,neighbourhood,municipality,play_demand,a_00_14,geometry,index_right,Unnamed: 0,id,name,addr:postcode,type,distance
0,Appingedam-Oost,Appingedam,7,900,POINT (6.859 53.307),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,0.046349
1,Hoogkerk-Zuid,Groningen,5,965,POINT (6.492 53.206),715,715,node/2739293160,Sporthal Hoogkerk,9744AP,nl_sports_centre,0.015526
2,Beijum-West,Groningen,6,1005,POINT (6.587 53.249),1879,1879,node/6452881273,Mauritshal,9785BG,nl_sports_centre,0.015510
3,Beijum-Oost,Groningen,6,1175,POINT (6.601 53.249),1879,1879,node/6452881273,Mauritshal,9785BG,nl_sports_centre,0.018492
4,Bedum,Het Hogeland,6,1205,POINT (6.600 53.302),1879,1879,node/6452881273,Mauritshal,9785BG,nl_sports_centre,0.038352
...,...,...,...,...,...,...,...,...,...,...,...,...
8342,Oud Gastel,Halderberge,5,810,POINT (4.463 51.584),9181,9181,node/7656252182,ARSV Thor,NaN,nl_track,0.075031
8343,Kalsdonk,Roosendaal,5,855,POINT (4.474 51.540),9181,9181,node/7656252182,ARSV Thor,NaN,nl_track,0.029953
8344,Klundert,Moerdijk,5,880,POINT (4.533 51.664),9181,9181,node/7656252182,ARSV Thor,NaN,nl_track,0.161084
8345,'s-Gravendeel Woonkern,Hoeksche Waard,6,1315,POINT (4.618 51.779),9181,9181,node/7656252182,ARSV Thor,NaN,nl_track,0.300320


In [9]:
joined['distance'] = round(joined['distance'] * 110.486, 2)
joined

,neighbourhood,municipality,play_demand,a_00_14,geometry,index_right,Unnamed: 0,id,name,addr:postcode,type,distance
0,Appingedam-Oost,Appingedam,7,900,POINT (6.859 53.307),1032,1032,node/2831440481,Manege Biesum,9931BT,nl_sports_centre,5.12
1,Hoogkerk-Zuid,Groningen,5,965,POINT (6.492 53.206),715,715,node/2739293160,Sporthal Hoogkerk,9744AP,nl_sports_centre,1.72
2,Beijum-West,Groningen,6,1005,POINT (6.587 53.249),1879,1879,node/6452881273,Mauritshal,9785BG,nl_sports_centre,1.71
3,Beijum-Oost,Groningen,6,1175,POINT (6.601 53.249),1879,1879,node/6452881273,Mauritshal,9785BG,nl_sports_centre,2.04
4,Bedum,Het Hogeland,6,1205,POINT (6.600 53.302),1879,1879,node/6452881273,Mauritshal,9785BG,nl_sports_centre,4.24
...,...,...,...,...,...,...,...,...,...,...,...,...
8342,Oud Gastel,Halderberge,5,810,POINT (4.463 51.584),9181,9181,node/7656252182,ARSV Thor,NaN,nl_track,8.29
8343,Kalsdonk,Roosendaal,5,855,POINT (4.474 51.540),9181,9181,node/7656252182,ARSV Thor,NaN,nl_track,3.31
8344,Klundert,Moerdijk,5,880,POINT (4.533 51.664),9181,9181,node/7656252182,ARSV Thor,NaN,nl_track,17.80
8345,'s-Gravendeel Woonkern,Hoeksche Waard,6,1315,POINT (4.618 51.779),9181,9181,node/7656252182,ARSV Thor,NaN,nl_track,33.18


In [10]:
joined.loc[joined['neighbourhood'] == 'Haren']

,neighbourhood,municipality,play_demand,a_00_14,geometry,index_right,Unnamed: 0,id,name,addr:postcode,type,distance
5,Haren,Groningen,6,1485,POINT (6.586 53.171),109,109,node/469191049,Groninger Kanovereniging,9752XJ,nl_sports_centre,1.52
492,Haren,Groningen,6,1485,POINT (6.586 53.171),2020,2020,node/2842128528,Gymzaal,9471SE,nl_sports_hall,13.02
986,Haren,Groningen,6,1485,POINT (6.586 53.171),2322,2322,node/2740107533,Het Helpernest,9721AJ,nl_kindergarten,2.97
1478,Haren,Groningen,6,1485,POINT (6.586 53.171),2970,2970,node/2748274225,Kinderopvang Groningen,9725AC,nl_childcare,4.35
1969,Haren,Groningen,6,1485,POINT (6.586 53.171),3490,3490,node/7335004779,NaN,NaN,nl_fitness_station,3.63
2461,Haren,Groningen,6,1485,POINT (6.586 53.171),3760,3760,node/1000113541,NaN,NaN,nl_pitch,3.77
2950,Haren,Groningen,6,1485,POINT (6.586 53.171),5125,5125,node/1357031613,Mytylschool Prins Johan Friso,NaN,nl_school,2.01
3441,Haren,Groningen,6,1485,POINT (6.586 53.171),7725,7725,node/4166301502,NaN,NaN,nl_swimming_pool,9.68
3932,Haren,Groningen,6,1485,POINT (6.586 53.171),7775,7775,node/2776714378,Korfbalvereniging SKF,NaN,nl_stadium,170.27
4429,Haren,Groningen,6,1485,POINT (6.586 53.171),7788,7788,node/649061908,Noorderpoort College Euroborg,9723ZS,nl_college,3.99


In [11]:
facilities.loc[facilities['name'] == 'Dreamfit']

,Unnamed: 0,id,name,addr:postcode,type,geometry
8089,8089,node/2740108333,Dreamfit,9721AJ,nl_fitness_centre,POINT (6.581 53.197)
